<a href="https://colab.research.google.com/github/ludomazza/ADM-HW1/blob/main/Copia_di_main_Albachiara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Homework 3 - Places of the world**

*Authors: Bellaroba Albachiara, Mazza Ludovica*

To get started, let's import all the libraries we will need.

In [1]:
from bs4 import BeautifulSoup
import bs4
import requests
import os.path
import lxml
import re
import codecs
import csv
import nltk
from datetime import datetime
from tqdm import tqdm
import time
import pandas as pd

In order to work in a faster environment, we link our work to a shared drive folder.

In [99]:
from google.colab import drive
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. Data collection

### 1.1. Get the list of places

At this point we're creating a text file with all the link related to the 7200 places we will analyze.

In [ ]:
linklist = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'w')

for i in tqdm(range(1, 401)):
    link = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
    cont = requests.get(link)
    soup = BeautifulSoup(cont.content, features='lxml')
    for a in soup.find_all('a', class_='content-card content-card-place'):
        linklist.write(a.get('href')+'\n')
        
linklist.close()

100%|██████████| 400/400 [02:05<00:00,  3.18it/s]


Control that we have all the link.

In [ ]:
file = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r')
count=0
for i in file:
  if i != '\n':
    count += 1
file.close
print(count)

7200


### 1.2. Crawl places

In [ ]:
def download(start, finish):
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for i in tqdm(range(start, finish)):
      cart = '/content/drive/MyDrive/ADM_HW3/Html_page/page_{}'.format(i)
      os.makedirs(cart)
      
      f = open('/content/drive/MyDrive/ADM_HW3/linklist.txt', 'r', encoding='utf8')
      l = f.readlines()[(i-1)*18:(i)*18]
      l = [k.rstrip() for k in l]
      f.close

      m = 1+18*(i-1)

      for link in l:
        link = 'https://www.atlasobscura.com' + link
        html = requests.get(link, headers = header)
        if html.status_code != 200:
          time.sleep(200)
          html = requests.get(link, headers = header)


        name = '{}/{}.html'.format(cart, m)
        text = open(name, 'w', encoding='utf8')
        text.write(html.text)
        text.close
        m=m+1


In [ ]:
download(1, 401)

100%|██████████| 100/100 [26:07<00:00, 15.67s/it]


### 1.3 Parse downloaded pages

In [ ]:
placeName=[]
placeTags=[]
numPeopleVisited=[]
numPeopleWant=[]
placeDesc=[]
placeShortDesc=[]
placeNearby=[]
placeAddress=[]
placeAlt=[]
placeLong=[]
placeEditors=[]
placePubDate=[]
placeRelatedLists=[]
placeRelatedPlaces=[]
placeURL=[]

In [ ]:

 for i in tqdm(range(1,401)):
   files = os.listdir('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i))
   for k in range(((18*(int(i)-1))+1), (18*int(i)+1)):
     file = open('/content/drive/MyDrive/ADM_HW3/Html_page/page_'+str(i)+'/'+str(k)+'.html', 'r', encoding='utf8')
     soup=bs4.BeautifulSoup(file, 'lxml')

     try:
       name=soup.find('h1', {'class':'DDPage__header-title'}).text
       placeName.append(name)
     except:
       placeName.append('')
         
     try:
       tag=soup.find_all('a', {'class':'itemTags__link js-item-tags-link'})
       tag=[i.get('href').split('/categories/')[1] for i in tag]
       placeTags.append(tag)
     except:
       placeTags.append('')
       
     try:
       visitors=soup.find_all('div', {'class':'title-md item-action-count'})[0].text.strip()
       numPeopleVisited.append(visitors)
     except:
       numPeopleVisited.append('')
    
     try:
       want=soup.find_all('div', {'class':'title-md item-action-count'})[1].text.strip()
       numPeopleWant.append(want)
     except:
       numPeopleWant.append('')

     try:
       desc=soup.find('div', {'class':'DDP__body-copy'}).text.strip()
       placeDesc.append(desc)
     except:
       placeDesc.append('')

     try:
       shdesc=soup.find('h3', {'class':'DDPage__header-dek'}).text.strip()
       placeShortDesc.append(shdesc)
     except:
       placeShortDesc.append('')

     try:
       nearby=soup.find_all('div', {'class':'DDPageSiderailRecirc__item-title'})
       nearby=[i.get_text() for i in nearby]
       placeNearby.append(nearby)
     except:
       placeNearby.append('')

     try:
       address=soup.find_all('address', {'class':'DDPageSiderail__address'})[0]
       address=address.find('div').get_text(separator=' ').split('\n')[0]
       placeAddress.append(address)
     except:
       placeAddress.append('')

     try:
       coordinates=soup.find_all('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})[0].text.strip()
       p=coordinates.split(', ')
       placeAlt.append(p[0])
       placeLong.append(p[1])
     except:
       placeAlt.append('')
       placeLong.append('')

     try:
       editors=soup.find_all('div', {'class':'ugc-editor-icons'})
       editors=[i.findChildren('a')[0].get('href') for i in editors]
       editors=[k.split('/users/')[1] for k in editors]
       placeEditors.append(editors)
     except:
       placeEditors.append('')

     try:
       date=soup.find('div', {'class':'DDPContributor__name'}).text.strip()
       placePubDate.append(date)
     except:
       placePubDate.append('')

     try:
       rellist=soup.find_all('div', {'data-gtm-template':'DDP Footer Recirc Lists'})[0]
       rellist=[i.text.strip() for i in rellist.find_all('h3')]
       placeRelatedLists.append(rellist)
     except:
       placeRelatedLists.append('')

     try:
       relplaces=soup.find_all('div', {'class':'CardRecircSection__title'})
       relplaces=[i for i in relplaces if i.text == 'Related Places'][0]
       relplaces=relplaces.find_all_next('div', {'class':'Card__action-btns vue-js-been-there-everywhere-place'})
       relplaces=[k.get('data-place-title') for k in relplaces]
       placeRelatedPlaces.append(relplaces)
     except:
       placeRelatedPlaces.append('')

     try:
       url=soup.find('meta', attrs={'property':'og:url'})['content']
       placeURL.append(url)
     except:
       placeURL.append('')




100%|██████████| 400/400 [48:41<00:00,  7.30s/it]


In [ ]:
features=pd.DataFrame(columns=['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 'placeDesc', 'placeShortDesc',
                                            'placeNearby', 'placeAddress', 'placeAlt', 'placeLong', 'placeEditors', 'placePubDate', 
                                            'placeRelatedLists', 'placeRelatedPlaces', 'placeURL'], index=[1,7200])

for i in tqdm(range(0, 7200)):
  features.at[i, 'placeName'] = placeName[i]
  features.at[i, 'placeTags'] = placeTags[i]
  features.at[i, 'numPeopleVisited']= numPeopleVisited[i]
  features.at[i, 'numPeopleWant']= numPeopleWant[i]
  features.at[i, 'placeDesc']= placeDesc[i]
  features.at[i, 'placeShortDesc']= placeShortDesc[i]
  features.at[i, 'placeNearby']= placeNearby[i]
  features.at[i, 'placeAddress']= placeAddress[i]
  features.at[i, 'placeAlt']= placeAlt[i]
  features.at[i, 'placeLong']= placeLong[i]
  features.at[i, 'placeEditors']= placeEditors[i]
  features.at[i, 'placePubDate']= placePubDate[i]
  features.at[i, 'placeRelatedLists']= placeRelatedLists[i]
  features.at[i, 'placeRelatedPlaces']= placeRelatedPlaces[i]
  features.at[i, 'placeURL']= placeURL[i]
  

100%|██████████| 7200/7200 [00:23<00:00, 303.84it/s]


In [ ]:
#os.makedirs('/content/drive/MyDrive/ADM_HW3/tsv_files')
placesfeatures = features.to_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', sep='\t', index=False)

In [59]:
visual=pd.read_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', sep='\t')
visual.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2625.0,8208.0,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['anthonyhogg'],"August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1834.0,8606.0,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'annetta-black']","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...
3,Leadenhall Market,"['markets', 'film-locations', 'film']",3142.0,7595.0,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'gavin-292130']","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...
4,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2430.0,7425.0,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'bylow1md']","November 21, 2008","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ
5,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4456.0,7057.0,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['cpilgrim', 'akazem']","February 13, 2009","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...
6,The Evolution Store,"['wonder-cabinets', 'purveyors-of-curiosities'...",2198.0,6951.0,Evolution stands out among the clothing stores...,A terrific purveyor of natural history objects...,"[""Merchant's House Museum"", 'The Brown Buildin...","687 Broadway New York, New York, 10012 United ...",40.7281,-73.9948,"['mbison', 'ramosju']","November 21, 2008","[""The World's Top 100 Wonders in 2018"", ""New Y...","['Kunstkammer Georg Laue', 'Ospedale delle Bam...",http://www.atlasobscura.com/places/evolution-n...
7,Magowan's Infinite Mirror Maze,"['optical-illusions', 'labyrinths', 'optical-o...",909.0,6507.0,"People stumble about the labyrinth, arms outst...",A psychedelic labyrinth on the San Francisco bay.,"['Musée Mécanique', 'Jack Early Park', ""Fior D...","Pier 39, Building O-11 San Francisco, Califor...",37.8100,-122.4104,"['guttersniperebecca', 'douglasthayer']","December 20, 2016","['24 Mesmerizing Mazes and Labyrinths', ""The W...","['The Winchester Mizmaze', 'Cathedral of Thorn...",http://www.atlasobscura.com/places/magowans-in...
8,Underground Tunnels of Los Angeles,"['tunnels', 'subterranean-sites', 'architectur...",660.0,6440.0,While the rest of the country was forced to go...,"During prohibition, corrupt city offi

In [ ]:
#come salvare file tsv

#with open(f'./all_Pages/{i}/{k}', encoding='utf8') as fp:
  #csv.writer(fp, delimiter = "\t").writerow([placeTag, placeName....])

## 2. Search Engine

In [3]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [39]:
# from tables.index import join_path
# def cleaning(line):
    
#     #filtered_line = []
#     stopWords = set(stopwords.words("english"))
#     punc =  set(string.punctuation) 
#     virgolette=['"', "'"]
#     newpunc=punc.union(virgolette)
#     wordTokens = word_tokenize(line)
#     ps = PorterStemmer()
    
#     filtered_line="".join([ps.stem(w) for w in wordTokens if (w not in stopWords) and (w not in string.punctuation)])
#     #filtered_line.append([ps.stem(w) for w in wordTokens if (w not in stopWords) and (w not in newpunc)])
    
#     return filtered_line

In [70]:
def clean(line):
  #split into token
  tokens = word_tokenize(line)
  #convert to lowercase
  tokens = [w.lower() for w in tokens]
  #remove punctuation
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  #filterout stopwords
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  #stemming
  ps = PorterStemmer()
  stem_sentence=[]

  for word in words:
    stem_sentence.append(ps.stem(word))

  return stem_sentence

In [97]:
desc_col=pd.read_csv('/content/drive/MyDrive/ADM_HW3/features.tsv', usecols=['placeDesc'] ,sep='\t')

In [98]:
desc_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7201 entries, 0 to 7200
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   placeDesc  7199 non-null   object
dtypes: object(1)
memory usage: 56.4+ KB


In [96]:
desc_col["placeDesc"] = desc_col["placeDesc"].map(lambda a: a.toString())

AttributeError: ignored

TypeError: ignored

In [92]:
desc_col=desc_col.apply(clean,axis=0)

AttributeError: ignored

### 2.1.1 Create your index!

### 2.1.2 Execute the query

### 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index

### 2.2.2 Execute the query

## 3. Define a new score!

## 4. Visualizing the most relevant places

## 5. BONUS: More complex search engine

## 6. Command line question

## 7. Theoretical question